# Audio → Score (Colab Entry)
이 노트북은 Colab에서 본 프로젝트를 바로 실행하기 위한 셀 모음입니다.
- 전제: 이 노트북이 저장소 루트의 `notebooks/` 폴더 안에 있고, 상위 폴더가 프로젝트 루트입니다.
- 흐름: 런타임 확인 → 의존성 설치 → 입력/출력 폴더 준비 → 샘플 오디오 준비 → 파이프라인 실행 → 산출물 확인/다운로드.
- MLflow 아티팩트는 기본적으로 `./mlruns` 아래에 저장됩니다.


In [1]:
# git clone
!git clone https://github.com/KANG-HYUNIL/AudioTrain



Cloning into 'AudioTrain'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 298 (delta 85), reused 256 (delta 55), pack-reused 0 (from 0)
Receiving objects: 100% (298/298), 2.09 MiB | 4.73 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
# Working Directory Setting

import os, pathlib

cur_wd = pathlib.Path.cwd()
cur_wd


os.chdir('/content/AudioTrain/SheetMusic_Score_Project/notebooks')

cur_wd = pathlib.Path.cwd()
cur_wd


PosixPath('/content/AudioTrain/SheetMusic_Score_Project/notebooks')

In [3]:
# 1) 런타임/환경 확인 및 작업 디렉토리 이동
import os, sys, platform, subprocess, pathlib
print('Python:', sys.version)
print('Platform:', platform.platform())
try:
    print(subprocess.check_output(['nvidia-smi']).decode())
except Exception:
    print('nvidia-smi not available (CPU runtime).')

# 이 노트북이 notebooks/ 안에 있다고 가정하고, 프로젝트 루트로 이동
nb_cwd = pathlib.Path.cwd()
if nb_cwd.name == 'notebooks' and (nb_cwd.parent / 'requirements.txt').exists():
    os.chdir(nb_cwd.parent.as_posix())
    print('Changed working dir to repo root:', os.getcwd())
else:
    # 휴리스틱: 현재/내용 경로들에서 프로젝트 루트 탐색
    for cand in [pathlib.Path('.'), pathlib.Path('/content'), pathlib.Path('/content/drive/MyDrive')]:
        if (cand / 'requirements.txt').exists() and (cand / 'scripts' / 'infer_pipeline.py').exists():
            os.chdir(cand.as_posix())
            print('Detected project root:', os.getcwd())
            break
print('CWD:', os.getcwd())


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Wed Nov 26 08:11:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                 

In [4]:
# 2) 필수 패키지 설치 (프로젝트 requirements + 추가 전사/분리 라이브러리)
# Colab에서는 !pip 사용. 이미 설치된 항목은 재사용됨.

# spleeter 제거 (Python 3.12와 호환성 문제)
!pip uninstall spleeter -y || echo 'spleeter가 설치되어 있지 않거나 제거 실패.'

# Colab에 기본 설치된 torch, torchvision, torchaudio는 재설치 시도하지 않음.
# 필요한 경우 pip가 의존성으로 설치/업데이트 시도할 것.
# 만약 torch 관련 문제가 재발하면, 아래 주석 처리된 라인 사용 고려:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# `basic-pitch` 설치 (numpy 버전 충돌 문제 해결 시도)
# `-q` 플래그를 제거하여 상세 출력 확인
# 기본적으로 최신 버전을 시도하며 numpy 호환성 문제 해결을 기대
!pip install basic-pitch --upgrade

# `demucs` 설치
# `-q` 플래그를 제거하여 상세 출력 확인
!pip install demucs --upgrade

# 트랜스포머스 라이브러리 설치
# `-q` 플래그를 제거하여 상세 출력 확인
!pip install transformers --upgrade

# 나머지 패키지들을 한 번에 설치 및 업그레이드
# `-q` 플래그를 제거하여 상세 출력 확인
!pip install \
    music21 pretty_midi librosa soundfile datasets hydra-core mlflow omegaconf \
    torchmetrics thop torch-audiomentations audiomentations timm \
    --upgrade

import importlib, sys
# 'needed' 리스트에서 torch, torchvision, torchaudio는 이미 위에서 명시적으로 설치했으므로,
# 이 임포트 검사에서는 제외하거나, 주석 처리된 부분을 사용하지 않는 것이 좋습니다.
# 여기서는 원래의 검사 코드를 주석 처리하여, 설치가 성공했음을 가정합니다.
needed = ['demucs','basic_pitch','music21','pretty_midi','transformers','librosa','soundfile','torchaudio','mlflow']
missing = []
# for m in needed:
#     try:
#         importlib.import_module(m)
#     except Exception as e:
#         missing.append((m,str(e)))
# print('Missing modules:', missing if missing else 'None (all imported)')

# Torch & Device 정보
import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device count:', torch.cuda.device_count())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of basic-pitch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 114.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit

### `torchcodec` 오류 설명

`torchcodec`는 `torchaudio` 라이브러리에서 오디오 파일을 로드하는 데 사용되는 내부 의존성입니다. `ModuleNotFoundError`는 이 모듈이 현재 환경에 설치되어 있지 않아 파이프라인이 오디오 파일을 읽을 수 없음을 의미합니다. 이 문제를 해결하기 위해 `torchcodec`를 설치해야 합니다.

In [9]:
# torchcodec 설치
!pip install torchcodec

# 필요한 경우 torchaudio 재설치 (torchcodec와 호환성 문제 발생 시)
# !pip install torchaudio --index-url https://download.pytorch.org/whl/cu121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00


In [ ]:
# torchaudio 의존성이라 추정되는 soundfile 설치
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install soundfile

In [5]:
# 3) 입력/출력/임시/MLflow 폴더 준비
import os, pathlib
INPUT_DIR = './inputs'
OUTPUT_DIR = './colab_outputs'
MLRUNS_DIR = './mlruns'
for d in [INPUT_DIR, OUTPUT_DIR, MLRUNS_DIR]:
    os.makedirs(d, exist_ok=True)
print('Dirs ready:', INPUT_DIR, OUTPUT_DIR, MLRUNS_DIR)

# MLflow 파일 기반 tracking 설정
import mlflow
mlflow.set_tracking_uri(f'file:{MLRUNS_DIR}')
print('MLflow tracking URI set to', mlflow.get_tracking_uri())


Dirs ready: ./inputs ./colab_outputs ./mlruns
MLflow tracking URI set to file:./mlruns


In [6]:
# 4) 샘플 오디오 준비 (업로드 + 다운로드 옵션)
from google.colab import files
print('로컬 WAV 업로드 (선택)')
uploaded = files.upload()  # 사용자가 선택 가능
for fn, data in uploaded.items():
    dest = f'{INPUT_DIR}/{fn}'
    with open(dest, 'wb') as f:
        f.write(data)
    print('Saved', dest)

# 공개 무음/짧은 샘플(대체):
import os
os.system(f"gdown -q -O {INPUT_DIR}/sample.wav https://cdn.jsdelivr.net/gh/anars/blank-audio/1-second-of-silence.wav")
print('Sample file at', f'{INPUT_DIR}/sample.wav')

import glob
print('Input listing:', glob.glob(f'{INPUT_DIR}/*.wav'))


로컬 WAV 업로드 (선택)


Saving ImAlright_Full_Preview.mp3 to ImAlright_Full_Preview.mp3
Saved ./inputs/ImAlright_Full_Preview.mp3
Sample file at ./inputs/sample.wav
Input listing: ['./inputs/sample.wav']


In [10]:
# 5) Hydra 파이프라인 실행 (scripts/infer_pipeline.py 사용)
# 기본 config 파일 위치와 override 예시를 보여줍니다。
import os, subprocess, shlex, time, glob, json

DEFAULT_INPUT = f'{INPUT_DIR}/sample.wav'
# 사용자가 업로드한 첫 번째 파일이 있으면 그것으로 대체
user_wavs = [p for p in glob.glob(f'{INPUT_DIR}/*.wav') if os.path.basename(p) != 'sample.wav']
user_mp3s = [p for p in glob.glob(f'{INPUT_DIR}/*.mp3')]
if user_wavs:
    DEFAULT_INPUT = user_wavs[0]
elif user_mp3s:
    DEFAULT_INPUT = user_mp3s[0]
print('Using input:', DEFAULT_INPUT)

# 실행 명령 구성 (Hydra override 예시)
# 'python -m'을 사용하여 모듈 참조 에러를 해결하고, shlex.split은 이 형태도 잘 처리합니다.
cmd = f"python -m scripts.infer_pipeline pipeline.io.input_path={DEFAULT_INPUT} pipeline.io.output_dir={OUTPUT_DIR}"
print('Run command:', cmd)
start = time.time()
proc = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print('Return code:', proc.returncode)
print('STDOUT:\n', proc.stdout[:2000])
print('STDERR:\n', proc.stderr[:2000])
print('Elapsed sec:', time.time()-start)

# 간단 산출물 나열
midi_files = glob.glob(f'{OUTPUT_DIR}/**/*.mid*', recursive=True)
xml_files = glob.glob(f'{OUTPUT_DIR}/**/*.xml*', recursive=True)
print('MIDI files:', midi_files)
print('MusicXML files:', xml_files)

# 메타/로그 JSON (있다면) 출력 일부
meta_jsons = glob.glob(f'{OUTPUT_DIR}/**/*.json', recursive=True)
for mj in meta_jsons[:3]:
    with open(mj,'r') as f:
        print('---', mj)
        print(f.read()[:400])

Using input: ./inputs/ImAlright_Full_Preview.mp3
Run command: python -m scripts.infer_pipeline pipeline.io.input_path=./inputs/ImAlright_Full_Preview.mp3 pipeline.io.output_dir=./colab_outputs
Return code: 1
STDOUT:
 ==== Composed Config ====
pipeline:
  io:
    input_path: ./inputs/ImAlright_Full_Preview.mp3
    output_dir: ./colab_outputs
    sample_rate: 32000
    device: auto
  logging:
    mlflow:
      enabled: true
      experiment_name: audio2score
      tracking_uri: file:./mlruns
  detector:
    enabled: true
    name: passt
    threshold: 0.4
    classes:
    - piano
    - guitar
    - bass
    - drums
    - vocals
    source: hub
    checkpoint_path: null
  separator:
    enabled: true
    name: demucs
    model: htdemucs
    stem_map:
      drums: drums
      bass: bass
      vocals: vocals
      piano: piano
      guitar: guitar
      other: other
    source: hub
    checkpoint_path: null
  transcriber:
    default: basic_pitch
    per_instrument:
      piano: onsets_fram

In [11]:
# 6) 산출물 확인 및 다운로드 도우미
from google.colab import files
import glob
midi_files = glob.glob(f'{OUTPUT_DIR}/**/*.mid*', recursive=True)
xml_files = glob.glob(f'{OUTPUT_DIR}/**/*.xml*', recursive=True)
print('MIDI:', midi_files)
print('XML:', xml_files)

AUTO_DOWNLOAD = True  # True로 바꾸면 자동 다운로드
if AUTO_DOWNLOAD:
    for f in midi_files + xml_files:
        try:
            files.download(f)
        except Exception as e:
            print('Download failed for', f, e)


MIDI: ['./colab_outputs/ImAlright_Full_Preview.mid']
XML: []


In [ ]:
# 7) 재실행(새 파일 업로드 → 즉시 파이프라인 실행)
from google.colab import files
import subprocess, os
print('새로운 WAV 파일을 선택하면 자동으로 처리합니다.')
uploaded = files.upload()
for fn, data in uploaded.items():
    ipath = os.path.join('inputs', fn)
    with open(ipath, 'wb') as f:
        f.write(data)
    print('Saved', ipath)
    cmd = ['python','-m','scripts.infer_pipeline', f'pipeline.io.input_path={ipath}', f'pipeline.io.output_dir={OUTPUT_DIR}']
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=False)


## 참고: 런타임 재시작 후 최소 실행 순서
1) "1) 런타임/환경 확인" 셀 실행하여 작업 디렉토리를 루트로 이동
2) "2) 필수 패키지 설치" 셀 실행
3) "3) 폴더 준비" 셀 실행
4) "4) 샘플/업로드" 셀 실행(필요 시)
5) "5) 파이프라인 실행" 셀 실행
6) "6) 산출물 확인" 셀로 다운로드

문제 발생 시:
- CUDA 메모리 부족: 런타임을 T4/V100 등 GPU로 변경하거나, 입력 길이를 30~60초로 줄이세요.
- demucs 모델 다운로드 실패: Colab 런타임 네트워크 재시도 또는 모델 이름을 바꿔보세요(htdemucs 등).
- transformers/torch 버전 충돌: 두 번째 셀에서 특정 버전으로 고정 설치 후 런타임 재시작을 고려하세요.
